Part 1 Data Preprocessing

In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

Importing the data

In [3]:
dataset = pd.read_csv('D:/PGP-BABI/Deep Learning with Krish Naik/Churn_Modelling.csv')

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Assigning Dependent and Independent Variables

In [15]:
x= dataset.iloc[:, 3:13]
y= dataset.iloc[: ,13]

Creating Dummy Variables for 

In [17]:
geography = pd.get_dummies(x["Geography"], drop_first= True)
gender = pd.get_dummies(x["Gender"], drop_first=True)

COoncatinating DUmmy Variables

In [18]:
x=pd.concat([x,geography,gender], axis=1)

In [19]:
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


Deleting the orginal geography and gender column

In [23]:
x=x.drop(['Geography','Gender'],axis=1)

In [24]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


Splitting the training and testing data

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2 , random_state =0)

C:\Users\Kiran Kumar\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Feature Scaling 

In [26]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

Building the Artificial Neural Network

Importing Keras and other relevant libraries

In [28]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

Initializing the ANN

In [29]:
classifier = Sequential()

Adding the input layer and the first hidden layer


In [30]:
classifier.add(Dense(units = 6, kernel_initializer= 'he_uniform', activation ='relu', input_dim = 11))

Instructions for updating:
Colocations handled automatically by placer.


Adding the second hidden layer

In [31]:
classifier.add(Dense(units=6, kernel_initializer= 'he_uniform', activation='relu'))

Adding the output layer

In [32]:
classifier.add(Dense(output_dim = 1, kernel_initializer='glorot_uniform', activation = 'sigmoid'))

C:\Users\Kiran Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="glorot_uniform", activation="sigmoid", units=1)`
  """Entry point for launching an IPython kernel.


In [33]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


Compiling the ANN

In [38]:
classifier.compile(optimizer='Adamax', loss= 'binary_crossentropy', metrics= ['accuracy'])

Fitting the ANN to the training set

In [40]:
model_history = classifier.fit(x_train,y_train, validation_split=0.33, batch_size=10, nb_epoch = 100)

C:\Users\Kiran Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Instructions for updating:
Use tf.cast instead.
Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 2s 289us/step - loss: 0.5151 - accuracy: 0.7962 - val_loss: 0.4933 - val_accuracy: 0.7955
Epoch 2/100
5359/5359 [==============================] - 1s 172us/step - loss: 0.4837 - accuracy: 0.7962 - val_loss: 0.4792 - val_accuracy: 0.7955
Epoch 3/100
5359/5359 [==============================] - 1s 161us/step - loss: 0.4695 - accuracy: 0.7962 - val_loss: 0.4669 - val_accuracy: 0.7955
Epoch 4/100
5359/5359 [==============================] - 1s 161us/step - loss: 0.4584 - accuracy: 0.7962 - val_loss: 0.4578 - val_accuracy: 0.7955
Epoch 5/100
5359/5359 [==============================] - 1s 158us/step - loss: 0.4497 - accuracy: 0.7962 - val_loss: 0.4503 - val_accuracy: 0.7955
Epoch 6/100
5359/5359 [==============================] - 1s 162us/step - loss: 0.4434 - accuracy: 0.7962 - val_loss: 0.4450 - val_accuracy: 0.7955
Epoch 7/100
5359/5359 

5359/5359 [==============================] - 1s 230us/step - loss: 0.3464 - accuracy: 0.8589 - val_loss: 0.3686 - val_accuracy: 0.8512
Epoch 55/100
5359/5359 [==============================] - 1s 174us/step - loss: 0.3460 - accuracy: 0.8584 - val_loss: 0.3668 - val_accuracy: 0.8482
Epoch 56/100
5359/5359 [==============================] - 1s 172us/step - loss: 0.3454 - accuracy: 0.8602 - val_loss: 0.3659 - val_accuracy: 0.8497
Epoch 57/100
5359/5359 [==============================] - 1s 170us/step - loss: 0.3446 - accuracy: 0.8597 - val_loss: 0.3663 - val_accuracy: 0.8497
Epoch 58/100
5359/5359 [==============================] - 1s 176us/step - loss: 0.3441 - accuracy: 0.8582 - val_loss: 0.3664 - val_accuracy: 0.8504
Epoch 59/100
5359/5359 [==============================] - 1s 169us/step - loss: 0.3437 - accuracy: 0.8593 - val_loss: 0.3654 - val_accuracy: 0.8508
Epoch 60/100
5359/5359 [==============================] - 1s 171us/step - loss: 0.3429 - accuracy: 0.8589 - val_loss: 0.3649 

Predicting the test set results

In [42]:
y_pred= classifier.predict(x_test)
y_pred = (y_pred>0.5)

Making the confusion Matrix


In [46]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[1532,   63],
       [ 216,  189]], dtype=int64)

Calcuting the accuracy

In [47]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
score

0.8605